In [82]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
from keras.preprocessing import sequence,text
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras import layers
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")



# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.tsv', 'test.tsv', 'sampleSubmission.csv']


In [83]:
train=pd.read_csv('../input/train.tsv',sep='\t')
print(train.shape)
train.head()

(156060, 4)


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [84]:
test=pd.read_csv('../input/test.tsv',sep='\t')
print(test.shape)
test.head()

(66292, 3)


,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [85]:
sub=pd.read_csv('../input/sampleSubmission.csv')
print(sub.shape)
sub.head()

(66292, 2)


,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2


In [86]:
test['Sentiment']=-100
test.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,156061,8545,An intermittently pleasing but mostly routine ...,-100
1,156062,8545,An intermittently pleasing but mostly routine ...,-100
2,156063,8545,An,-100
3,156064,8545,intermittently pleasing but mostly routine effort,-100
4,156065,8545,intermittently pleasing but mostly routine,-100


In [87]:
df=pd.concat([train,test],ignore_index=True)
print(df.shape)
df.tail()

(222352, 4)


,PhraseId,SentenceId,Phrase,Sentiment
222347,222348,11855,"A long-winded , predictable scenario .",-100
222348,222349,11855,"A long-winded , predictable scenario",-100
222349,222350,11855,"A long-winded ,",-100
222350,222351,11855,A long-winded,-100
222351,222352,11855,predictable scenario,-100


In [88]:
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.stem import SnowballStemmer,WordNetLemmatizer
lemma=WordNetLemmatizer()
from string import punctuation
import re
stemmer=SnowballStemmer('english')

In [89]:
def review_cleaner(review_col):
    corpus=[]
    for i in range (0,len(review_col)):
        review=str(review_col[i])
        review=re.sub('[^a-zA-Z]',' ',review)
        review=[lemma.lemmatize(w) for w in word_tokenize(str(review).lower())]
        review=' '.join(review)
        corpus.append(review)
    return corpus

In [90]:
df['clean_review']=review_cleaner(df.Phrase.values)
df.head()

,PhraseId,SentenceId,Phrase,Sentiment,clean_review
0,1,1,A series of escapades demonstrating the adage ...,1,a series of escapade demonstrating the adage t...
1,2,1,A series of escapades demonstrating the adage ...,2,a series of escapade demonstrating the adage t...
2,3,1,A series,2,a series
3,4,1,A,2,a
4,5,1,series,2,series


In [91]:
df_train=df[df.Sentiment!=-100]
df_test=df[df.Sentiment==-100]
df_test.drop('Sentiment',axis=1,inplace=True)
print(df_train.shape,df_test.shape)
df_test.head()
del df

(156060, 5) (66292, 4)


In [92]:
df_test.head()

,PhraseId,SentenceId,Phrase,clean_review
156060,156061,8545,An intermittently pleasing but mostly routine ...,an intermittently pleasing but mostly routine ...
156061,156062,8545,An intermittently pleasing but mostly routine ...,an intermittently pleasing but mostly routine ...
156062,156063,8545,An,an
156063,156064,8545,intermittently pleasing but mostly routine effort,intermittently pleasing but mostly routine effort
156064,156065,8545,intermittently pleasing but mostly routine,intermittently pleasing but mostly routine


In [93]:
train_text=df_train.clean_review.values
test_text=df_test.clean_review.values
target=df_train.Sentiment.values
y=to_categorical(target)
print(train_text.shape,target.shape,y.shape)

(156060,) (156060,) (156060, 5)


In [94]:
X_train_text,X_val_text,y_train,y_val=train_test_split(train_text,y,test_size=0.2,stratify=y,random_state=123)
print(X_val_text.shape,y_val.shape)
print(X_train_text.shape,y_train.shape)

(31212,) (31212, 5)
(124848,) (124848, 5)


In [95]:
all_words=' '.join(X_train_text)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
num_unique_word=len(dist)
num_unique_word


13732

In [96]:
r_len=[]
for t in X_train_text:
    w=word_tokenize(t)
    l=len(w)
    r_len.append(l)
max_len=np.max(r_len)
print(max_len)
    

48


In [97]:
features=num_unique_word
max_words=max_len
batch_size=512
epochs=5
num_classes=5

In [98]:
tokenizer=Tokenizer(num_words=features)
tokenizer.fit_on_texts(list(X_train_text))
X_train=tokenizer.texts_to_sequences(X_train_text)
X_val=tokenizer.texts_to_sequences(X_val_text)
X_test=tokenizer.texts_to_sequences(test_text)


In [99]:
X_train=sequence.pad_sequences(X_train, maxlen=max_words)
X_val=sequence.pad_sequences(X_val, maxlen=max_words)
X_test=sequence.pad_sequences(X_test, maxlen=max_words)
print(X_test.shape,X_val.shape,X_train.shape)

(66292, 48) (31212, 48) (124848, 48)


In [100]:
model1=Sequential()
model1.add(layers.Embedding(features,100,mask_zero=True))
model1.add(layers.LSTM(64,dropout=0.4,recurrent_dropout=0.4,return_sequences=True))
model1.add(layers.LSTM(32,dropout=0.4,recurrent_dropout=0.4,return_sequences=False))
model1.add(layers.Dense(num_classes,activation='softmax'))
model1.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.002),metrics=['accuracy'])
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 100)         1373200   
_________________________________________________________________
lstm_5 (LSTM)                (None, None, 64)          42240     
_________________________________________________________________
lstm_6 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 165       
Total params: 1,428,021
Trainable params: 1,428,021
Non-trainable params: 0
_________________________________________________________________


In [101]:
history1=model1.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1)

Train on 124848 samples, validate on 31212 samples
Epoch 1/5
124848/124848 [==============================] - 67s 534us/step - loss: 1.1482 - acc: 0.5550 - val_loss: 0.8963 - val_acc: 0.6382
Epoch 2/5
124848/124848 [==============================] - 64s 513us/step - loss: 0.8436 - acc: 0.6569 - val_loss: 0.8368 - val_acc: 0.6584
Epoch 3/5
124848/124848 [==============================] - 64s 513us/step - loss: 0.7775 - acc: 0.6805 - val_loss: 0.8236 - val_acc: 0.6636
Epoch 4/5
124848/124848 [==============================] - 64s 512us/step - loss: 0.7401 - acc: 0.6926 - val_loss: 0.8198 - val_acc: 0.6662
Epoch 5/5
124848/124848 [==============================] - 64s 509us/step - loss: 0.7100 - acc: 0.7041 - val_loss: 0.8238 - val_acc: 0.6662


In [102]:
y_pred1=model1.predict_classes(X_test)

In [103]:
sub.Sentiment=y_pred1
sub.to_csv('sub1.csv',index=False)
sub.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2


In [106]:
model2=Sequential()
model2.add(layers.Embedding(features,100,input_length=max_words))
model2.add(layers.Dropout(0.3))
model2.add(layers.Conv1D(128,kernel_size=3,padding='same',activation='relu',strides=1))
model2.add(layers.GlobalMaxPooling1D())
model2.add(layers.Dense(128,activation='relu'))
model2.add(layers.Dropout(0.3))
model2.add(layers.Dense(num_classes,activation='softmax'))

model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 48, 100)           1373200   
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 100)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 48, 128)           38528     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 645       
Total para

In [107]:
history2=model2.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 124848 samples, validate on 31212 samples
Epoch 1/5
124848/124848 [==============================] - 7s 53us/step - loss: 1.1335 - acc: 0.5568 - val_loss: 0.9375 - val_acc: 0.6259
Epoch 2/5
124848/124848 [==============================] - 2s 17us/step - loss: 0.8547 - acc: 0.6518 - val_loss: 0.8609 - val_acc: 0.6467
Epoch 3/5
124848/124848 [==============================] - 2s 17us/step - loss: 0.7757 - acc: 0.6804 - val_loss: 0.8302 - val_acc: 0.6559
Epoch 4/5
124848/124848 [==============================] - 2s 18us/step - loss: 0.7260 - acc: 0.6986 - val_loss: 0.8142 - val_acc: 0.6598
Epoch 5/5
124848/124848 [==============================] - 2s 17us/step - loss: 0.6876 - acc: 0.7119 - val_loss: 0.8138 - val_acc: 0.6587


In [108]:
y_pred2=model2.predict_classes(X_test)

In [109]:
sub.Sentiment=y_pred2
sub.to_csv('sub2.csv',index=False)
sub.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2


In [110]:
model3= Sequential()
model3.add(layers.Embedding(features,100,input_length=max_words))
model3.add(layers.Conv1D(128,kernel_size=3,padding='same',activation='relu'))
model3.add(layers.MaxPooling1D(pool_size=2))
model3.add(layers.Dropout(0.2))
model3.add(layers.GRU(256,return_sequences=True))
model3.add(layers.Dropout(0.3))
model3.add(layers.Flatten())
model3.add(layers.Dense(128,activation='relu'))
model3.add(layers.Dropout(0.3))
model3.add(layers.Dense(num_classes,activation='softmax'))
model3.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.002),metrics=['accuracy'])
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 48, 100)           1373200   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 48, 128)           38528     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 24, 128)           0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 24, 128)           0         
_________________________________________________________________
gru_1 (GRU)                  (None, 24, 256)           295680    
_________________________________________________________________
dropout_4 (Dropout)          (None, 24, 256)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6144)              0         
__________

In [111]:
history3=model3.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size)

Train on 124848 samples, validate on 31212 samples
Epoch 1/5
124848/124848 [==============================] - 15s 117us/step - loss: 1.0418 - acc: 0.5861 - val_loss: 0.8498 - val_acc: 0.6563
Epoch 2/5
124848/124848 [==============================] - 12s 97us/step - loss: 0.7989 - acc: 0.6734 - val_loss: 0.8018 - val_acc: 0.6720
Epoch 3/5
124848/124848 [==============================] - 12s 98us/step - loss: 0.7084 - acc: 0.7053 - val_loss: 0.8093 - val_acc: 0.6704
Epoch 4/5
124848/124848 [==============================] - 12s 98us/step - loss: 0.6528 - acc: 0.7232 - val_loss: 0.8133 - val_acc: 0.6728
Epoch 5/5
124848/124848 [==============================] - 12s 98us/step - loss: 0.6004 - acc: 0.7414 - val_loss: 0.8674 - val_acc: 0.6671


In [112]:
y_pred3=model3.predict_classes(X_test)

In [113]:
sub.Sentiment=y_pred3
sub.to_csv('sub3.csv',index=False)
sub.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2


In [116]:
model4 = Sequential()
model4.add(layers.Embedding(features, 100, input_length=max_words))
model4.add(layers.SpatialDropout1D(0.3))
model4.add(layers.Bidirectional(layers.GRU(256)))
model4.add(layers.Dropout(0.3))
model4.add(layers.Dense(num_classes, activation='softmax'))
model4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 48, 100)           1373200   
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 48, 100)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               548352    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 5)                 2565      
Total params: 1,924,117
Trainable params: 1,924,117
Non-trainable params: 0
_________________________________________________________________


In [117]:
history4=model4.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size)

Train on 124848 samples, validate on 31212 samples
Epoch 1/5
124848/124848 [==============================] - 44s 353us/step - loss: 1.0791 - acc: 0.5700 - val_loss: 0.8893 - val_acc: 0.6355
Epoch 2/5
124848/124848 [==============================] - 41s 326us/step - loss: 0.8312 - acc: 0.6602 - val_loss: 0.8326 - val_acc: 0.6620
Epoch 3/5
124848/124848 [==============================] - 41s 325us/step - loss: 0.7604 - acc: 0.6856 - val_loss: 0.8139 - val_acc: 0.6701
Epoch 4/5
124848/124848 [==============================] - 41s 325us/step - loss: 0.7148 - acc: 0.7033 - val_loss: 0.8189 - val_acc: 0.6712
Epoch 5/5
124848/124848 [==============================] - 41s 329us/step - loss: 0.6800 - acc: 0.7144 - val_loss: 0.8215 - val_acc: 0.6698


In [118]:
y_pred4=model4.predict_classes(X_test)

In [119]:
sub.Sentiment=y_pred4
sub.to_csv('sub4.csv',index=False)
sub.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2
